In [ ]:
from simterfere import simulation, visibility, wavefront
import numpy as np
from astropy.io import fits
import multiprocessing as mp
from scipy.ndimage import gaussian_filter
from scipy.optimize import least_squares
import os

In [ ]:
# Load ft and sc wavelength bins
wave_ft = np.loadtxt('../data/gravity_wavelengths_ft.txt')
wave_sc = np.loadtxt('../data/gravity_wavelengths_sc_highres.txt')

# For simplicity we use an uniform ft and sc flux 
flux_ft = np.ones_like(wave_ft)
flux_sc = np.ones_like(wave_sc)

In [ ]:
# Use ideal FT V2PM
V2PM_ideal = np.array([
            [1,1,0,0,2,0,0,0,0,0],
            [1,1,0,0,2j,0,0,0,0,0],
            [1,1,0,0,-2,0,0,0,0,0],
            [1,1,0,0,-2j,0,0,0,0,0],

            [1,0,1,0,0,2,0,0,0,0],
            [1,0,1,0,0,2j,0,0,0,0],
            [1,0,1,0,0,-2,0,0,0,0],
            [1,0,1,0,0,-2j,0,0,0,0],

            [1,0,0,1,0,0,2,0,0,0],
            [1,0,0,1,0,0,2j,0,0,0],
            [1,0,0,1,0,0,-2,0,0,0],
            [1,0,0,1,0,0,-2j,0,0,0],

            [0,1,1,0,0,0,0,2,0,0],
            [0,1,1,0,0,0,0,2j,0,0],
            [0,1,1,0,0,0,0,-2,0,0],
            [0,1,1,0,0,0,0,-2j,0,0],

            [0,1,0,1,0,0,0,0,2,0],
            [0,1,0,1,0,0,0,0,2j,0],
            [0,1,0,1,0,0,0,0,-2,0],
            [0,1,0,1,0,0,0,0,0-2j,0],

            [0,0,1,1,0,0,0,0,0,2],
            [0,0,1,1,0,0,0,0,0,2j],
            [0,0,1,1,0,0,0,0,0,-2],
            [0,0,1,1,0,0,0,0,0,-2j],
                 ],dtype=np.complex128)

V2PM_FT = np.repeat(V2PM_ideal[:, :, np.newaxis], len(wave_ft), axis=2)

In [ ]:
# Get the SC V2PM from the calibration files
file_v2pm = "../data/V2PM/GRAVI.2024-12-20T10_49_26.376_p2vm.fits"

indices = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]
data_v2pm = fits.open(file_v2pm)[4].data
tm0 = data_v2pm["TRANSMISSION"]
coh0 = data_v2pm["COHERENCE"]
phase0 = data_v2pm["PHASE"]
cm0 = data_v2pm["C_MATRIX"]
tm0 = np.abs(tm0)
wave_v2pm = fits.open(file_v2pm)[3].data["EFF_WAVE"]*1e6
nwave = len(wave_v2pm)

V2PM0 = np.empty((24,10,nwave),dtype=np.complex128)
V2PM0[:,:4] = tm0
V2PM0[:,4:] = coh0*np.exp(1j*phase0)

# We have to resoort the matrix to match with the definition in SIMTERFERE
V2PM = np.empty((24,10,nwave),dtype=np.complex128)
V2PM[:4] = V2PM0[8:12]
V2PM[4:8] = V2PM0[20:]
V2PM[8:12] = V2PM0[4:8]
V2PM[12:16] = V2PM0[16:20]
V2PM[16:20] = V2PM0[:4]
V2PM[20:] = V2PM0[12:16]

cm = np.empty((24,6,nwave))
cm[:4] = cm0[8:12]
cm[4:8] = cm0[20:]
cm[8:12] = cm0[4:8]
cm[12:16] = cm0[16:20]
cm[16:20] = cm0[:4]
cm[20:] = cm0[12:16]

In [18]:
# In the C-matrix only the throughput for the six coherent fluxes is saved. 
# To get the flux throughput we have to solve the overdetermined system Fij = sqrt(Fi*Fj) 

def residuals(F, V):
    F4, F3, F2, F1 = F
    V_model = np.array([
        np.sqrt(F4 * F3),
        np.sqrt(F4 * F2),
        np.sqrt(F4 * F1),
        np.sqrt(F3 * F2),
        np.sqrt(F3 * F1),
        np.sqrt(F2 * F1)
    ])
    return V_model - V

def fit_flux(i, cmcoh_slice):
    F0 = np.ones(4)
    result = least_squares(residuals, F0, args=(cmcoh_slice,), bounds=(0, np.inf))
    F_out = result.x

    return i, F_out

tm = np.real(V2PM[:,:4])
coh = np.abs(V2PM[:,4:])
phase = np.angle(V2PM[:,4:])

# Prepare input slices
cm_gaus = gaussian_filter(cm,(0,0,10)) #We filter the throughput to remove absorption lines
args = [(i, np.mean(cm_gaus[:,:,i],axis=0)) for i in range(nwave)]

# Allocate result array
F_fit = np.empty((4, nwave))

# Run in parallel to speed it up
with mp.Pool(processes=mp.cpu_count()-1) as pool:
    for i, result in pool.starmap(fit_flux, args):
        F_fit[:, i] = result

C_fit = np.empty((6,nwave))
for i in range(6):
    C_fit[i] = np.sqrt(F_fit[indices[i][0]]*F_fit[indices[i][1]])

V2PM_SC = np.empty((24,10,nwave),dtype=np.complex128)
V2PM_SC[:,:4] = F_fit[None]*tm
V2PM_SC[:,4:] = C_fit[None]*coh*np.exp(1j*phase)

In [ ]:
# Set up the simulation
file_path = "../data/tutorial/GRAVI.2024-12-20T00:51:57.856.fits" # Simulate coherent fluxes for this file
sky_path = "../data/tutorial/GRAVI.2024-12-20T00:53:21.860.fits" # Corresponding sky data
wavefront_path = '../wavefronts/tutorial/' #Save wavefronts here
path = '../simulations/turorial/' #Save coherent fluxes here

if not os.path.exists(wavefront_path):
    os.makedirs(wavefront_path)
if not os.path.exists(path):
    os.makedirs(path)

sim = simulation.Simulation(file_path,sky_path,wavefront_path,wave_sc,wave_ft,flux_sc,flux_ft)

# Fit wavefronts
sim.get_wavefronts()

# Simulate coherent fluxes with default settings
sim.simulate_visibilities(
         path=path,
         V2PM_SC=V2PM_SC,
         V2PM_SC_calib=V2PM_SC,
         wave_V2PM_SC=wave_v2pm,
         V2PM_FT=V2PM_FT,
         V2PM_FT_calib=V2PM_FT,
         wave_V2PM_FT=wave_ft,
         )